# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.
The dataset used was developed by the GroupLens group and can be found at the link : https://grouplens.org/datasets/movielens/ .
The use of the dataset was for educational purposes only.

In [1]:
import os
import glob
import pandas as pd
import sql_queries
from sql_queries import *
import mysql.connector as db_connect
from mysql.connector import Error
import mysql.connector 
import re

In [2]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.csv'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `movies_data`
In this first part, you'll perform ETL on the first and second dataset, `movies.csv` and `links.csv`, to create the `movies` dimensional tables.

Let's perform ETL on a movies csv file.
- Use the `get_files` function provided above to get a list of all song csv files in `ml-latest-small`
- Select the first and second files
- Read the movies and links file and view the data

In [3]:
movies_files = get_files('ml-latest-small')

In [4]:
links_csv = movies_files[0]
movies_csv = movies_files[1]
ratings_csv = movies_files[2]
tags_csv = movies_files[3]

In [5]:
df = pd.read_csv(movies_csv) 
df = df.rename(columns={"movieId":"movie_id"})

Show the extracted data

In [6]:
df.head()

,movie_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
df['year'] = df['title'].str[-6:]
df['year'] = df['year'].str.replace(r"[()]","")
df

,movie_id,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,2017
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,2017
9739,193585,Flint (2017),Drama,2017
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,2018


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  9742 non-null   int64 
 1   title     9742 non-null   object
 2   genres    9742 non-null   object
 3   year      9742 non-null   object
dtypes: int64(1), object(3)
memory usage: 190.3+ KB


In [11]:
df.describe()

,movie_id
count,9742.000000
mean,42200.353623
std,52160.494854
min,1.000000
25%,3248.250000
50%,7300.000000
75%,76232.000000
max,193609.000000


In [12]:
df_temp = df.drop_duplicates(subset=['title'])

In [13]:
df_temp.describe()

,movie_id
count,9737.000000
mean,42165.340865
std,52136.321521
min,1.000000
25%,3247.000000
50%,7294.000000
75%,76173.000000
max,193609.000000


In [15]:
df.loc[df['movie_id'] == 3265]

,movie_id,title,genres,year
2452,3265,Hard-Boiled (Lat sau san taam) (1992),Action|Crime|Drama|Thriller,1992


Read the link file and extract data

In [17]:
df_links = pd.read_csv(links_csv)
df_links = df_links.rename(columns={"movieId":"movie_id", 'imdbId':'imdb_id','tmdbId':'tmdb_id'})
df_links

,movie_id,imdb_id,tmdb_id
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


Merge the two dataframes

In [18]:
df_movies = pd.merge(df, df_links, on="movie_id")
df_movies

,movie_id,title,genres,year,imdb_id,tmdb_id
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,114709,862.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,113497,8844.0
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,113228,15602.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,114885,31357.0
4,5,Father of the Bride Part II (1995),Comedy,1995,113041,11862.0
...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,2017,5476944,432131.0
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,2017,5914996,445030.0
9739,193585,Flint (2017),Drama,2017,6397426,479308.0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,2018,8391976,483455.0


In [19]:
movies_data = list(df_movies[['movie_id', 'title', 'genres', 'year', 'imdb_id', 'tmdb_id']].values[0])
movies_data = df_movies[['movie_id', 'title', 'genres', 'year', 'imdb_id', 'tmdb_id']]
movies_data

,movie_id,title,genres,year,imdb_id,tmdb_id
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,114709,862.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,113497,8844.0
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,113228,15602.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,114885,31357.0
4,5,Father of the Bride Part II (1995),Comedy,1995,113041,11862.0
...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,2017,5476944,432131.0
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,2017,5914996,445030.0
9739,193585,Flint (2017),Drama,2017,6397426,479308.0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,2018,8391976,483455.0


#### Insert movie into Movies Table
this section implements and tests the insertion in the `movies table` of the database `movies`. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `movies` table in the database.

In [ ]:
db = "movies" 
pw = "440y58ttw"
#pw ="yourPassword"
connection = mysql.connector.connect(
            host="localhost",
            user="root",
            passwd=pw,
            database='movies',
            auth_plugin = 'mysql_native_password'
        )

cursor = connection.cursor()

In [ ]:

#sql = "INSERT INTO movies (movie_id, title, genres, year, imdb_id, tmdb_id) VALUES (%s, %s, %s, %s, %s, %s)"
#cursor.execute(sql, movies_data)
df_erro = pd.DataFrame()
for i, row in df_movies.iterrows():
    try:
        # Executing the SQL command
        cursor.execute(movies_table_insert, list(row))
        connection.commit()
    except:
        # Rolling back in case of error
        connection.rollback()
        df_erro = df_erro.append(row)
        print("Erro | ")
df_erro

In [48]:

sql = "DELETE FROM movies"
cursor.execute(sql)
connection.commit()
sql = "DELETE FROM ratings"
cursor.execute(sql)
connection.commit()
sql = "DELETE FROM tags"
cursor.execute(sql)
connection.commit()



In [49]:
sql = "SELECT * FROM movies"
cursor.execute(sql)

myresult = cursor.fetchall()

for x in myresult:
  print(x)

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [23]:
df_ratings = pd.read_csv(ratings_csv)
df_ratings = df_ratings.rename(columns={"userId":"user_id", "movieId":"movie_id"})
df_ratings = df_ratings.drop("timestamp", axis = 1)
df_ratings

,user_id,movie_id,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [24]:
for i, row in df_ratings.iterrows():
    try:
        # Executing the SQL command
        cursor.execute(ratings_table_insert, list(row))
        connection.commit()
    except:
        # Rolling back in case of error
        connection.rollback()
        print("Erro")

In [38]:
sql = "SELECT * FROM ratings"
cursor.execute(sql)

myresult = cursor.fetchall()

for x in myresult:
  print(x)

In [ ]:
df_tags = pd.read_csv(tags_csv)
df_tags = df_tags.rename(columns={"userId":"user_id", "movieId":"movie_id"})
df_tags = df_tags.drop("timestamp", axis = 1)
df_tags

,user_id,movie_id,tag
0,2,60756,funny
1,2,60756,Highly quotable
2,2,60756,will ferrell
3,2,89774,Boxing story
4,2,89774,MMA
...,...,...,...
3678,606,7382,for katie
3679,606,7936,austere
3680,610,3265,gun fu
3681,610,3265,heroic bloodshed


In [50]:
df_erro2 = pd.DataFrame()
for i, row in df_tags.iterrows():
    try:
        # Executing the SQL command
        cursor.execute(tags_table_insert, list(row))
        connection.commit()
    except:
        # Rolling back in case of error
        connection.rollback()
        df_erro2 = df_erro2.append(row)
        print("Erro | ")
df_erro2

Erro | 
Erro | 
Erro | 
Erro | 
Erro | 
Erro | 
Erro | 
Erro | 
Erro | 
Erro | 
Erro | 
Erro | 
Erro | 
Erro | 
Erro | 
Erro | 
Erro | 


,movie_id,tag,user_id
212,63992.0,audience intelligence underestimated,62.0
229,87430.0,audience intelligence underestimated,62.0
485,52287.0,Something for everyone in this one... saw it w...,161.0
587,48698.0,the catholic church is the most corrupt organi...,318.0
621,364.0,Oscar (Best Music - Original Score),319.0
1201,912.0,start of a beautiful friendship,474.0
1699,2919.0,Academy award (Best Supporting Actress),474.0
2379,8879.0,Oscar (Best Supporting Actress),474.0
2433,31658.0,06 Oscar Nominated Best Movie - Animation,474.0
2465,37729.0,06 Oscar Nominated Best Movie - Animation,474.0


In [39]:
sql = "SELECT * FROM tags"
cursor.execute(sql)

myresult = cursor.fetchall()

for x in myresult:
  print(x)

In [40]:
db = "movies" 
pw = "440y58ttw"
#pw ="yourPassword"
connection = mysql.connector.connect(
            host="localhost",
            user="root",
            passwd=pw,
            database='movies',
            auth_plugin = 'mysql_native_password'
        )

cursor = connection.cursor()